# 🎨 Studio Ghibli Art Generator

This notebook allows you to transform your images into Studio Ghibli-style artwork using Stable Diffusion.

## Installation

First, let's install the required dependencies:

In [ ]:
!pip install torch torchvision torchaudio diffusers transformers accelerate safetensors pillow

## Import Libraries

In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import io
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

## Load the Model

In [ ]:
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float32
    ).to(device)
    return pipe

pipe = load_model()

## Create UI Elements

In [ ]:
# Create widgets
upload = widgets.FileUpload(
    accept='.jpg,.jpeg,.png',
    multiple=False,
    description='Upload Image:'
)

strength_slider = widgets.FloatSlider(
    value=0.75,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Strength:',
    continuous_update=False
)

guidance_slider = widgets.FloatSlider(
    value=7.5,
    min=0.0,
    max=20.0,
    step=0.5,
    description='Guidance:',
    continuous_update=False
)

prompt_text = widgets.Textarea(
    value='A beautiful Studio Ghibli-style landscape with vibrant colors and soft lighting',
    description='Prompt:',
    layout=widgets.Layout(width='100%')
)

transform_button = widgets.Button(
    description='Transform Image',
    button_style='success'
)

# Create output area
output = widgets.Output()

## Display UI and Handle Transformations

In [ ]:
def on_transform_button_clicked(b):
    with output:
        clear_output()
        
        if not upload.value:
            print("Please upload an image first!")
            return
            
        # Get the uploaded image
        uploaded_file = list(upload.value.values())[0]
        image_bytes = uploaded_file['content']
        
        # Convert to PIL Image
        init_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        init_image = init_image.resize((512, 512))
        
        # Display original image
        print("Original Image:")
        display(init_image)
        
        print("Generating Ghibli-style image...")
        
        # Generate image
        generator = torch.manual_seed(42)
        output_image = pipe(
            prompt=prompt_text.value,
            image=init_image,
            strength=strength_slider.value,
            guidance_scale=guidance_slider.value
        ).images[0]
        
        # Display result
        print("Generated Image:")
        display(output_image)
        
        # Save the image
        output_image.save("ghibli_output.png")
        print("\nImage saved as 'ghibli_output.png'")

In [ ]:
# Connect the button to the function
transform_button.on_click(on_transform_button_clicked)

# Display the UI
display(widgets.VBox([
    upload,
    strength_slider,
    guidance_slider,
    prompt_text,
    transform_button,
    output
]))

## Usage Instructions

1. Upload an image using the file uploader
2. Adjust the settings:
   - **Strength**: Controls how much the AI modifies the image (0.0 to 1.0)
   - **Guidance**: Controls how closely the AI follows the prompt (0.0 to 20.0)
   - **Prompt**: Customize the style description
3. Click "Transform Image" to generate the Ghibli-style version
4. The generated image will be displayed and saved as 'ghibli_output.png'

### Tips for Best Results:
- Use high-quality images for better results
- Adjust the transformation strength to control how much the image changes
- Try different prompts to achieve different styles
- Higher guidance values will make the AI follow the prompt more closely

### Note:
The first run will download the Stable Diffusion model, which may take a few minutes.